In [7]:
import pandas as pd
import tqdm
import matplotlib.pyplot as plt

## Import Yelp Data

In [11]:
local = False

In [14]:
data_path = r"C:\Users\Willi\Documents\NYU\2020 Spring\Big Data Science\Project\yelp_dataset/yelp_academic_dataset_"

business_fname = data_path + 'business.json'
# checkin_fname = data_path + 'checkin.json'
# photo_fname = data_path + 'photo.json'
if local:
    review_fname = data_path + 'review_short.json'
else:
    review_fname = data_path + 'review.json'
# tip_fname = data_path + 'tip.json'
# user_fname = data_path + 'user.json'

biz_df = pd.read_json(business_fname, lines=True)
print("business json attributes: ", list(biz_df.columns.values))

# ci_df = pd.read_json(checkin_fname, lines=True)
# print("checkin json attributes: ", list(ci_df.columns.values))

# ph_df = pd.read_json(photo_fname, lines=True)
# print("photo json attributes: ", list(ph_df.columns.values))

rev_df = pd.read_json(review_fname, lines=True)
print("review json attributes: ", list(rev_df.columns.values))

# tip_df = pd.read_json(tip_fname, lines=True)
# print("tip json attributes: ", list(tip_df.columns.values))

# use_df = pd.read_json(user_fname, lines=True)
# print("user json attributes: ", list(use_df.columns.values))

business json attributes:  ['address', 'attributes', 'business_id', 'categories', 'city', 'hours', 'is_open', 'latitude', 'longitude', 'name', 'postal_code', 'review_count', 'stars', 'state']


MemoryError: 

## Find Number of Empty Entries

In [ ]:
print("business empty entries: ", biz_df.isna().sum(axis=0))
print("review empty entries: ", rev_df.isna().sum(axis=0))

# print("checkin empty entries: ", ci_df.isna().sum(axis=0))
# print("photo empty entries: ", ph_df.isna().sum(axis=0))
# print("tip empty entries: ", tip_df.isna().sum(axis=0))
# print("user empty entries: ", use_df.isna().sum(axis=0))

## Sizes

In [ ]:
print("business size: ", biz_df.shape)
print("review size: ", rev_df.shape)

# print("checkin size: ", ci_df.shape)
# print("photo size: ", ph_df.shape)

# print("tip size: ", tip_df.shape)
# print("user size: ", use_df.shape)

## Set If Restaurant

In [ ]:
biz_df['is_restaurant'] = (biz_df.loc[:,'categories'].where(biz_df.loc[:,'categories']\
                                                                 .str.contains('Restaurant')).isna()==False)

In [ ]:
print(biz_df.loc[biz_df.loc[:,'is_restaurant']==False,'categories'])

In [ ]:
biz_df.head()


In [ ]:
rev_df.head()

### Business Categories

In [ ]:
def plotDict(valuesDict, topK, fileName, title):
    labels = list(valuesDict.keys())[0:topK]
    values = list(valuesDict.values())[0:topK]

    fig, axes = plt.subplots(figsize=(12, 10))
    axes.pie(values,
            labels = labels,
            
            startangle=90)
#     plt.figsize=(20, 20)
    
    plt.title(title)

    axes.axis('equal')
    plt.savefig(fileName)


In [ ]:
import collections

In [ ]:
categoryCount = {}

for businessCategories in biz_df.categories:
    if businessCategories is None:
        continue
    for category in businessCategories.split(','):
        category = category.strip().lower()
        if category in categoryCount:
            categoryCount[category] = categoryCount[category] + 1
        else:
            categoryCount[category] = 1
            
print ("Total Number of Categories : " + str(len(categoryCount)))
print ("Total Number of Businesses : " + str(biz_df.shape[0]))
sortedCategoryCountsDict = collections.OrderedDict({k: v for k, v in sorted(categoryCount.items(), key=lambda item: item[1], reverse = True)})
plotDict(sortedCategoryCountsDict, 10, "top10_business_categories.png", "Top-10 Business categories\n")

### Most Reviewed Business Categories 

In [ ]:
def updateCount(businessCategories, categoryCount, count):
    if businessCategories is None:
        return
    if review_count == 0:
        return
    for category in businessCategories.split(','):
        category = category.strip().lower()
        if category in categoryCount:
            categoryCount[category] = categoryCount[category] + count
        else:
            categoryCount[category] = count

In [ ]:
#Cross check this data against review count from business.json.

# categoryToReviewCount = {}
# for businessId, count in rev_df['business_id'].value_counts().iteritems():

#     for categories in biz_df.loc[biz_df['business_id'] == businessId]['categories']:
#         if categories is None:
#             continue
#         for category in categories.split(","):
#             if category is None:
#                 continue
#             category = category.strip().lower()
#             if category in categoryToReviewCount:
#                 categoryToReviewCount[category]  = categoryToReviewCount[category] + count
#             else:
#                 categoryToReviewCount[category] = count
# sortedcategoryToReviewCount = collections.OrderedDict(
#     {k: v for k, v in sorted(categoryToReviewCount.items(), key=lambda item: item[1], reverse = True)})

# plotDict(sortedcategoryToReviewCount, 10, 
#          "top10_business_by_reviews_categories.png", "Top-10 most reviewed business categories\n")


categoryReviewCounting = {}

for i in range (0, biz_df.shape[0]):
    row = biz_df.iloc[i]
    updateCount(row['categories'], categoryReviewCounting , row['review_count'])

sortedcategoryToReviewCount = collections.OrderedDict(
    {k: v for k, v in sorted(categoryReviewCounting.items(), key=lambda item: item[1], reverse = True)})

plotDict(sortedcategoryToReviewCount, 10, 
         "top10_business_by_reviews_categories.png", "Top-10 most reviewed business categories\n")

### Review Star Counting

In [ ]:
starCounts = {}
for stars, counts in rev_df['stars'].value_counts().iteritems():
    starCounts[stars] = counts

plt.bar(starCounts.keys(), starCounts.values(), color='b')
plt.title("Count by Star Ratings")
plt.savefig("starCount.png")

### Review Length Bins

In [ ]:
plt.hist(rev_df['text'].str.len(), bins = 50, color = "skyblue")
plt.xlabel("Review length")
plt.ylabel("Count")
plt.title("Review length histogram")
plt.savefig("review_length_bins.png")

### Missing Review Data

In [ ]:
print("Missing Review data\n")
print(rev_df.isna().sum())
print("\n\nMissing Business data\n")
print(biz_df.isna().sum())


## Summarize statistics

In [ ]:
# stats_index = ['Mean', 'Median', '1Q', '3Q', 'Std', 'Min', 'Max']

# biz_cols = ['review_count', 'stars']
# rev_cols = ['cool', 'funny', 'stars', 'useful']
# use_cols =  ['average_stars', 'compliment_cool', 'compliment_cute', 'compliment_funny', 'compliment_hot', 'compliment_list', 'compliment_more', 'compliment_note', 'compliment_photos', 'compliment_plain', 'compliment_profile', 'compliment_writer', 'cool', 'fans', 'funny', 'review_count', 'useful']

# biz_stats = pd.DataFrame(index = stats_index, columns = biz_cols)
# rev_stats = pd.DataFrame(index = stats_index, columns = rev_cols)
# use_stats = pd.DataFrame(index = stats_index, columns = use_cols)

# stats = [biz_stats, rev_stats, use_stats]
# dfs = [biz_df, rev_df, use_df]

# for i, stat in enumerate(stats):
#     temp = dfs[i]
#     for col in tqdm.tqdm_notebook(list(stat.columns.values)):
#         print(col)
#         stat.loc['Mean',col] = temp.loc[:,col].mean()
#         # stat.loc['Mode',col] = temp.loc[:,col].mode(dropna=True)
#         stat.loc['Median',col] = temp.loc[:,col].median()        
#         stat.loc['1Q',col] = temp.loc[:,col].quantile(q=0.25)
#         stat.loc['3Q',col] = temp.loc[:,col].quantile(q=0.75)
#         stat.loc['Std',col] = temp.loc[:,col].std()
#         stat.loc['Min',col] = temp.loc[:,col].min()
#         stat.loc['Max',col] = temp.loc[:,col].max()

In [8]:
# overview_cols = ['Number of Entries', 'Number of Incomplete', 'Number Attributes', 'Attributes']
# overview_inds = ['business', 'checkin', 'photo', 'review', 'tip', 'user']
# overview_dfs = [biz_df, ci_df, ph_df, rev_df, tip_df, use_df]

# overview = pd.DataFrame(index = overview_inds, columns = overview_cols)

# for i, df in enumerate(overview_dfs):
#     overview.iloc[i,0] = df.shape[0]
#     overview.iloc[i,1] = sum(df.isna().sum(axis=1)>0)
#     overview.iloc[i,2] = df.shape[1]
#     overview.iloc[i,3] = list(df.columns.values)

In [9]:
# biz_df.loc[:,'is_restaurant'].sum()

59387

In [10]:
# sum(biz_df.loc[biz_df.loc[:,'is_restaurant'],:].isna().sum(axis=1)>0)

15273

In [1]:
# rev_df.join(biz_df[('business_id','is_restaurant')],on='business_id',how='left')

NameError: name 'rev_df' is not defined

## Print Statistics

In [11]:
# save_path = r"C:\Users\Willi\Documents\NYU\2020 Spring\Big Data Science\Project\Data_files\Yelp Data Overview\\"

# overview_path = save_path + 'overview_stats.csv'
# biz_path = save_path + 'business_stats.csv'
# rev_path = save_path + 'review_stats.csv'
# use_path = save_path + 'user_stats.csv'

# stats_names = [biz_path, rev_path, use_path]

# overview.to_csv(overview_path)

# for i,stat in enumerate(stats):
#     stat.to_csv(stats_names[i])